In [10]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from bayes_opt import BayesianOptimization

In [2]:
data, target = make_classification(n_samples=10000,n_features=45,n_informative=12,n_redundant=7,random_state=2018)

In [11]:
dtrain = lgb.Dataset(data=data,label=target,free_raw_data=False)

In [9]:
lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt',is_unbalance=True,
                             colsample_bytree=0.7,max_depth=3,
                             learning_rate=0.1,min_child_weight=1.0,
                             objective='binary',
                             num_leaves=32,n_estimators=2000,
                             reg_alpha=0.0,reg_lambda=0.0,
                             subsample=0.9,random_state=2018)

In [42]:
def lgb_evaluate_cv(colsample_bytree=0.7,
                    learning_rate=0.1,num_leaves=32,
                    subsample=0.9,reg_alpha=0.0,
                    reg_lambda=0.0,min_child_weight=0.0):
    params = dict()
    params['colsample_bytree'] = max(min(colsample_bytree,1),0)
    params['learning_rate'] = max(min(learning_rate,1),0)
    # 贝叶斯优化过程，num_leaves会按照float类型计算，在实际lgb计算过程中，必须以int形式输入
    params['num_leaves'] = int(num_leaves)
    params['subsample'] = max(min(subsample,1),0)
    params['reg_alpha'] = max(0,reg_alpha)
    params['reg_lambda'] = max(0,reg_lambda)
    params['min_child_weight'] = int(min_child_weight)
    params['is_unbalance'] = True
    cv_result =  lgb.cv(params=params,train_set=dtrain,
                        early_stopping_rounds=20,
                        metrics='auc',nfold=5,
                        num_boost_round=2000,
                        verbose_eval=False,seed=1,show_stdv=True)                       
    return cv_result['auc-mean'][-1]

In [43]:
lgbBO = BayesianOptimization(f=lgb_evaluate_cv,
                             pbounds={'colsample_bytree':(0.7,0.7),
                                      'learning_rate':(0.01,0.2),
                                      'num_leaves':(7,31),
                                      'subsample':(0.7,0.95),
                                      'reg_alpha':(0.0,0.0),
                                      'reg_lambda':(0,0.0),
                                      'min_child_weight':(1,1)})

In [44]:
lgbBO.maximize(init_points=5,n_iter=5)

Initialization
---------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   min_child_weight |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 
    1 | 00m02s |    0.98588 |             0.7000 |          0.1928 |             1.0000 |      15.6744 |      0.0000 |       0.0000 |      0.8635 | 
    2 | 00m05s |    0.98721 |             0.7000 |          0.0956 |             1.0000 |      25.3274 |      0.0000 |       0.0000 |      0.7160 | 
    3 | 00m06s |    0.98652 |             0.7000 |          0.1636 |             1.0000 |      14.4048 |      0.0000 |       0.0000 |      0.8127 | 
    4 | 00m03s |    0.98705 |             0.7000 |          0.1456 |             1.0000 |      26.1786 |      0.0000 |       0.0000 |      0.8004 | 
    5 | 00m06s |    0.98577 |             0.7000 |          0.1984 |             1.0000 |   

D:\Program Files (x86)\anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00011974]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
---------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   min_child_weight |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 
    6 | 00m19s |    0.98607 |             0.7000 |          0.1900 |             1.0000 |       7.0095 |      0.0000 |       0.0000 |      0.7189 | 
    7 | 00m58s |    0.98854 |             0.7000 |          0.0182 |             1.0000 |      30.9980 |      0.0000 |       0.0000 |      0.7057 | 
    8 | 00m21s |    0.98633 |             0.7000 |          0.1999 |             1.0000 |      30.9147 |      0.0000 |       0.0000 |      0.7040 | 
    9 | 00m55s |    0.98525 |             0.7000 |          0.0110 |             1.0000 |      10.0637 |      0.0000 |       0.0000 |      0.7026 | 
   10 | 01m33s |    0.98842 |             0.7000 |          0.0108 |             1.00